In [1]:
import h5py # .h5 파일을 읽기 위한 패키지
import random
import pandas as pd
import numpy as np
import os
import glob
import math

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from tqdm.auto import tqdm
from cfg.voxelnet_cfg import config as cfg 

from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings(action='ignore') 

In [2]:
import torch.nn as nn
import torch.nn.functional as F
import torch


# conv2d + bn + relu
class Conv2d(nn.Module):
    def __init__(self, in_channels, out_channels, k,s,p, activation=True, batch_norm=True):
        super(Conv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=k,stride=s,padding=p)
        
        if batch_norm:
            self.bn = nn.BatchNorm2d(out_channels)
        else:
            self.bn = None
        self.activation = activation
        
    def forward(self,x):
        x = self.conv(x)
        if self.bn is not None:
            x=self.bn(x)
        if self.activation:
            return F.relu(x,inplace=True)
        else:
            return x

# conv3d + bn + relu
class Conv3d(nn.Module):

    def __init__(self, in_channels, out_channels, k, s, p, batch_norm=True):
        super(Conv3d, self).__init__()
        self.conv = nn.Conv3d(in_channels, out_channels, kernel_size=k, stride=s, padding=p)
        if batch_norm:
            self.bn = nn.BatchNorm3d(out_channels)
        else:
            self.bn = None

    def forward(self, x):
        x = self.conv(x)
        if self.bn is not None:
            x = self.bn(x)

        return F.relu(x, inplace=True)

# Fully Connected Network
class FCN(nn.Module):

    def __init__(self,cin,cout):
        super(FCN, self).__init__()
        self.cout = cout
        self.linear = nn.Linear(cin, cout)
        self.bn = nn.BatchNorm1d(cout)

    def forward(self,x):
        # KK is the stacked k across batch
#         print("1. FCN x.shape : ",x.shape)
        kk, t, _ = x.shape
#         N, D, H, W = x.shape

        x = self.linear(x.view(kk * t, -1))
#         x = self.linear(x.view(N*D*H, -1))
        

#         print("2. FCN x.shape : ", x.shape)
        x = F.relu(self.bn(x))
        
        return x.view(kk,t, -1)
#         return x.view(N, -1, H, x.shape[1])

# Voxel Feature Encoding layer
class VFE(nn.Module):

    def __init__(self,cin,cout):
        super(VFE, self).__init__()
        assert cout % 2 == 0
        self.units = cout // 2
        self.fcn = FCN(cin,self.units)

    def forward(self, x, mask):
        # point-wise feauture
        pwf = self.fcn(x)
        
        #locally aggregated feature
#         print("1. VEF pwf.shape : ", pwf.shape)
#         print("1. VEF torch max pwf shape : ",  torch.max(pwf,2)[0].shape)
#         print("1. VEF torch max pwf unsq(1) shape : ",  torch.max(pwf,2)[0].unsqueeze(2).shape)
#         print("1. VEF torch max pwf unsq(1) repeat(1,35,1) shape : ",  torch.max(pwf,2)[0].unsqueeze(2).repeat(1,1,cfg.T,1).shape)
        laf = torch.max(pwf,1)[0].unsqueeze(1).repeat(1,cfg.T,1)
#         laf = torch.max(pwf,2)[0].unsqueeze(2).repeat(1,1,cfg.T,1)
#         print("2. VEF laf.shape : ", laf.shape)
        
        # point-wise concat feature
        pwcf = torch.cat((pwf,laf),dim=2)
#         pwcf = torch.cat((pwf,laf),dim=3)
#         print("3. VEF pwcf.shape : ", pwcf.shape)

        # apply mask
#         print("4. VEF mask shape : ", mask.shape)
#         print("4. VEF mask unsq(2) shape : ", mask.unsqueeze(2).shape)
#         print("4. VEF mask unsq(2) repeat(1,1, ??) shape : ", mask.unsqueeze(2).repeat(1, 1, self.units * 2).shape)
        mask = mask.unsqueeze(2).repeat(1, 1, self.units * 2)
#         mask = mask.unsqueeze(3).repeat(1, 1, 1, self.units * 2)
#         print("4. VEF mask.shape : ", mask.shape)
        pwcf = pwcf * mask.float()

        return pwcf

# Stacked Voxel Feature Encoding
class SVFE(nn.Module):

    def __init__(self):
        super(SVFE, self).__init__()
#         self.vfe_1 = VFE(6,32)
        self.vfe_1 = VFE(7,32)
        self.vfe_2 = VFE(32,128)
        self.fcn = FCN(128,128)
        
    def forward(self, x):
        mask = torch.ne(torch.max(x,2)[0], 0)
#         print("SVFE Mask value : ", mask)
        x = self.vfe_1(x, mask)
        x = self.vfe_2(x, mask)
        x = self.fcn(x)
#         print("SVFE x.shape: ",x.shape)
        # element-wise max pooling
        x = torch.max(x,1)[0]
        return x

# Convolutional Middle Layer
class CML(nn.Module):
    def __init__(self):
        super(CML, self).__init__()
        self.conv3d_1 = Conv3d(128, 64, 3, s=(2, 1, 1), p=(1, 1, 1))
        self.conv3d_2 = Conv3d(64, 64, 3, s=(1, 1, 1), p=(0, 1, 1))
        self.conv3d_3 = Conv3d(64, 64, 3, s=(2, 1, 1), p=(1, 1, 1))

    def forward(self, x):
        x = self.conv3d_1(x)
        x = self.conv3d_2(x)
        x = self.conv3d_3(x)
        return x

class RPN(nn.Module):
    def __init__(self):
        super(RPN, self).__init__()
        self.block_1 = [Conv2d(192, 192, 3, 2, 1)]
        self.block_1 += [Conv2d(192, 192, 3, 1, 1) for _ in range(3)]
        self.block_1 = nn.Sequential(*self.block_1)

        self.block_2 = [Conv2d(192, 192, 3, 2, 1)]
        self.block_2 += [Conv2d(192, 192, 3, 1, 1) for _ in range(5)]
        self.block_2 = nn.Sequential(*self.block_2)

        self.block_3 = [Conv2d(192, 256, 3, 2, 1)]
        self.block_3 += [nn.Conv2d(256, 256, 3, 1, 1) for _ in range(5)]
        self.block_3 = nn.Sequential(*self.block_3)

        self.deconv_1 = nn.Sequential(nn.ConvTranspose2d(256, 256, 4, 4, 0),nn.BatchNorm2d(256))
        self.deconv_2 = nn.Sequential(nn.ConvTranspose2d(192, 256, 2, 2, 0),nn.BatchNorm2d(256))
        self.deconv_3 = nn.Sequential(nn.ConvTranspose2d(192, 256, 1, 1, 0),nn.BatchNorm2d(256))

        self.score_head = Conv2d(768, 10, 1, 1, 0, activation=False, batch_norm=False)
#         self.reg_head = Conv2d(768, 7 * cfg.anchors_per_position, 1, 1, 0, activation=False, batch_norm=False)
        self.cls = Classifier(cin=640)
    def forward(self,x, batch_size):
        x = self.block_1(x)
        x_skip_1 = x
        x = self.block_2(x)
        x_skip_2 = x
        x = self.block_3(x)
#         print("RPN block_3 x.shape : ",x.shape)
        x_0 = self.deconv_1(x)
#         print("RPN deconv_1 x_0.shape : ",x_0.shape)
        x_1 = self.deconv_2(x_skip_2)
#         print("RPN deconv_2 x_1.shape : ",x_1.shape)
        x_2 = self.deconv_3(x_skip_1)
#         print("RPN deconv_3 x_2.shape : ",x_2.shape)
        x = torch.cat((x_0,x_1,x_2),1)
#         print("RPN cat x.shape : ",x.shape)
        x = self.score_head(x)
#         print("RPN score_head x.shape : ",x.shape)
        x = self.cls(x.view(batch_size, -1))
#         print("RPN cls x.shape : ",x.shape)
        return x #self.score_head(x),self.reg_head(x)
    
    
# classifier
class Classifier(nn.Module):

    def __init__(self, cin, num_classes=10, dropout_rate=0.5):
        super(Classifier, self).__init__()  
        
        self.linear_1 = nn.Sequential(
            nn.Linear(cin, cin//2),
            nn.GELU(),
            nn.Dropout(p=dropout_rate, inplace=False)
        )
        
        self.linear_2 = nn.Sequential(
            nn.Linear(cin//2, cin//4),
            nn.GELU(),
            nn.Dropout(p=dropout_rate, inplace=False)
        )
        
        self.linear_3 = nn.Linear(cin//4, num_classes)


    def forward(self,x):
        # KK is the stacked k across batch
#         print("classifier x.shape : ", x.shape)
#         print("classifier x.view.shape : ", x.view(2,-1).shape)
        b, f, *_ = x.shape
        x = self.linear_1(x.view(b, -1))
        x = self.linear_2(x)
        x = self.linear_3(x)
        return x
    

class VoxelNet(nn.Module):

    def __init__(self):
        super(VoxelNet, self).__init__()
        self.svfe = SVFE()
        self.cml = CML()
        self.rpn = RPN()
        self.cls = Classifier(cin=64)
        
    def voxel_indexing(self, sparse_features, coords, batch_size):
        dim = sparse_features.shape[-1]
#         print("sparse_features.shape : ", sparse_features.shape)
        
        coords= coords.type(torch.long)
#         print("coords.shape : ", coords.shape)
#         print("coords values [:,0]: ",coords[:,0])
        dense_feature = torch.zeros(dim, batch_size, cfg.D, cfg.H, cfg.W).to(cfg.device)
#         print("dense_feature.shape : ", dense_feature.shape)
        dense_feature[:, coords[:,0], coords[:,1], coords[:,2], coords[:,3]]= sparse_features.transpose(0,1)
#         dense_feature[:, coords[:,0], coords[:,1], coords[:,2]]= sparse_features
        return dense_feature.transpose(0, 1)

    def forward(self, voxel_features, voxel_coords, batch_size):

        # feature learning network
        vwfs = self.svfe(voxel_features)
        vwfs = self.voxel_indexing(vwfs,voxel_coords, batch_size)

        # convolutional middle network
        cml_out = self.cml(vwfs)
        cml_out = cml_out.view(batch_size, -1,cfg.H, cfg.W)
#         print("cml_out.shape : ", cml_out.shape)
        # region proposal network
        score = self.rpn(cml_out, batch_size)
        # merge the depth and feature dim into one, output probability score map and regression map
        # psm,rm = self.rpn(cml_out.view(cfg.N,-1,cfg.H, cfg.W))
        
        # classifier
#         print("score shape : ", score)

        return score #psm, rm

In [3]:
def detection_collate(batch, train=True):
    voxel_features = []
    voxel_coords = []
    label = []
    
    for i, sample in enumerate(batch):
        voxel_features.append(sample[0])
        voxel_coords.append(np.pad(sample[1], ((0,0), (1,0)), mode='constant', constant_values=i))
            
    if train :
        return np.concatenate(voxel_features), np.concatenate(voxel_coords), batch[:,2]
    else :
        return np.concatenate(voxel_features), np.concatenate(voxel_coords), len(batch)
        

In [4]:
class VoxelDataset(Dataset) :
    def __init__(self, id_list, label_list, point_list) :
        self.id_list = id_list
        self.label_list = label_list
        self.point_list = point_list
    
    def __getitem__(self, index) :
        image_id = self.id_list[index]
        
        points= self.point_list[str(image_id)][:]
        voxel_features, voxel_coords= self.voxelization(points)

        if self.label_list is not None:
            label = self.label_list[index]
            return torch.Tensor(voxel_features), torch.Tensor(voxel_coords), label
        else:
            return torch.Tensor(voxel_features), torch.Tensor(voxel_coords)
    
    def voxelization(self, point) :
        point_reflectance = np.zeros((point.shape[0],1))
        point = np.concatenate((point, point_reflectance), 1)
        
        voxel_coords = ((point[:, :3] - np.array([cfg.xrange[0], cfg.yrange[0], cfg.zrange[0]])) / 
                       (cfg.vw, cfg.vh, cfg.vd)).astype(np.int32)
        
        # convert to (D,H,W)
        voxel_coords = voxel_coords[:, [2,1,0]]
        voxel_coords, inv_ind, voxel_counts = np.unique(voxel_coords, 
                                                        axis=0, 
                                                        return_inverse=True, 
                                                        return_counts=True)
        
        
        voxel_features = []
        for i in range(len(voxel_coords)) :
            voxel = np.zeros((cfg.T, 7), dtype=np.float32)
            pts = point[inv_ind == i]
            if voxel_counts[i] > cfg.T :
                pts = pts[:cfg.T, : ]
                voxel_counts[i] = cfg.T
            
            voxel[:pts.shape[0], :] = np.concatenate((pts, pts[:, :3] - np.mean(pts[:, :3], 0)), axis=1)
            voxel_features.append(voxel)

        return np.array(voxel_features), voxel_coords
    
    def __len__(self) :
        return len(self.id_list)

In [10]:
def predict(model, test_loader, device, dim_changer=None):
    model.to(device)
    model.eval()
    model_preds = []
    with torch.no_grad():
        for vf, vc, batch_size in tqdm(iter(test_loader)):
            vf = torch.tensor(vf).to(device)
            vc = torch.tensor(vc).to(device)

            batch_pred = model(vf, vc, batch_size=batch_size)
            
            model_preds += batch_pred.argmax(1).detach().cpu().numpy().tolist()
    
    return model_preds

In [6]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

test_df = pd.read_csv('./data/sample_submission.csv')
test_points = h5py.File('./data/test.h5', 'r')

test_dataset = VoxelDataset(test_df['ID'].values, None, test_points)
test_loader = DataLoader(test_dataset, batch_size = cfg.N,  collate_fn=detection_collate, shuffle=True, num_workers=0)

checkpoint = torch.load('./3E_best_model.pth')
model = VoxelNet().to(device)
model.load_state_dict(checkpoint)

<All keys matched successfully>

In [11]:
preds = predict(model, test_loader, device)

  0%|          | 0/625 [00:00<?, ?it/s]

In [12]:
test_df['label'] = preds

test_df.to_csv('./submission/3E_voxelnet.csv', index=False)